# Pickprops & Filter

Notebook to analyze 'locs_picked.hdf5' file in order to get 'locs_picked_props.hdf5' file containing kinetic parameters according to lbFCS and qPAINT analysis. Kinetic filter will be applied automatically. Resulting file can be dragged and dropped into picasso.filter module.

![](../../docs/figures/pickprops&filter.png)


### Define paths
Define paths to data, i.e. '_locs_picked.hdf5' as obtained by picasso.render module. Multiple files can be processed by extending the list.

In [27]:
dir_names=[]
dir_names.extend(['/fs/pool/pool-schwille-paint/Data/p12.ACAB/19-08-07_id143+144/114_Pm2_5nM_p35uW_1/19-08-07_PS'])

file_names=[]
file_names.extend(['114_Pm2_5nM_p35uW_1_MMStack_Pos0.ome_locs_render_picked.hdf5']) 

### Define input variables
Define input variables for lbFCS and qPAINT analysis and data saving. For evaluation of multiple files concentration list has to be extended.

In [28]:
conc=[5] # Imager concentration [nM]
ignore=1 # Ignore_dark value for qPAINT analysis
savename_ext='_props_ig%i'%(ignore) # File extension for processed file

#### Advanced
omit_dist=True # If True all lists will be excluded for saving (recommended) 
kin_filter=True # If True automatic filtering will be applied (recommended)
NoPartitions=30 # Number of partitions for dask parallel computing

### Run pickprops & filter

In [29]:
#################################################### Load packages
import os #platform independent paths
import importlib
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

# Load user defined functions
import lbfcs.pickprops as props
import lbfcs.io as io
import lbfcs.pickprops_calls as props_call
# Reload modules
importlib.reload(props)
importlib.reload(props_call)

############################################################# Read locs, apply props & save locs
#### Create list of paths
path=[os.path.join(dir_names[i],file_names[i]) for i in range(0,len(file_names))]

#### Dictonary added content for info '.yaml' file
props_info={'Generated by':'pickprops.get_props',
            'ignore':ignore,
            'omit_dist':omit_dist,
            'kin_filter': kin_filter}

#### Read-Apply-Save loop
for i in range(0,len(path)):
    #### File read in
    print('File read in ...')
    locs,locs_info=io.load_locs(path[i])
    
    #### Get number of frames
    NoFrames=locs_info[0]['Frames']
    
    #### Apply non-parallelized props
    print('Calculating kinetics ...')
#    locs_props=props.apply_props(locs,conc[i],NoFrames,ignore)
    
    #### Apply parallelized props
    locs_props=props.apply_props_dask(locs,conc[i],NoFrames,ignore,NoPartitions)
    
    #### Drop objects for saving if omit=True
    if omit_dist:
        print('Removing distribution-lists from output ...')
        locs_props=locs_props.drop(['trace','tau','g','tau_b_dist','tau_d_dist'],axis=1)
    
    if kin_filter:
        print('Applying kinetic filter ...')
        locs_props=props._kin_filter(locs_props)
    
    #### Add nearest neigbour pick and distance
    print('Calculating nearest neighbour ...')
    locs_props=props_call.props_add_nn(locs_props)
    
    #### Save .hdf5 and .yaml of locs_props    
    io.save_locs(path[i].replace('.hdf5',savename_ext+'.hdf5'),
                    locs_props,
                    [locs_info,props_info],
                    mode='picasso_compatible')

File read in ...
Calculating kinetics ...
[########################################] | 100% Completed | 27.4s
Removing distribution-lists from output ...
Applying kinetic filter ...


  0%|          | 0/6110 [00:00<?, ?it/s]

Calculating nearest neighbour ...


100%|██████████| 6110/6110 [00:10<00:00, 566.85it/s]
